In [ ]:
import numpy as np
import pandas as pd
import datetime 
from statsmodels.stats.moment_helpers import corr2cov
import scipy.stats
import bqplot as bq
import ipywidgets as widgets


## The data shows asset prices are random from moment to moment, or at least can be modeled that way. 
## Lets look at a stochastic process 

## $$ dS_t = \mu S_t dt + \sigma S_t dW_t$$


## Using ito's calculus if we solve this equation for any $S_0$ we get the value of $S_t$ at any point in the future with:

## $$ S_t = S_0  exp \left( \left( \mu - \frac{\sigma ^2}{2} \right) t + \sigma W_t \right)$$

In [ ]:
def VestaSim(I):
    
    S = S0 * np.exp(np.cumsum((mu - 0.5 * sigma ** 2) * dt
                + sigma * np.sqrt(dt) * np.random.standard_normal((M + 1, I)), axis=0))
    S[0] = S0
    return S

In [ ]:

S0 = 100
mu = 0.01
sigma = 0.02
T = 5.0; M = 60; dt = T / M    

nsims = 1

S = VestaSim(nsims)

In [ ]:
sc_x = bq.LinearScale()
sc_y = bq.LinearScale()

line = bq.Lines(x=np.arange(len(S)), y=S.transpose(),
             scales={'x': sc_x, 'y': sc_y})
ax_x = bq.Axis(scale=sc_x, label='Index')
ax_y = bq.Axis(scale=sc_y, orientation='vertical', label='Values')

Rando = bq.Figure(marks=[line], axes=[ax_x, ax_y], title='Random Values')

In [ ]:
Rando

## What happens if we put more than one asset into a portfolio? 
## lets start with 2 assets

Our Expected Return should be the weight of one asset's return plus the weight of the second asset's return:
## $$ E[r] = w\mu_1 + (1- w)\mu_2$$

## Our 2 asset portfolio variance is a little more complicated because the assets move $\sigma$ but they also move together $\rho$ (correlation)

## $$ Variance =  w_1^2\sigma_1^2 + w_2^2\sigma_2^2 + 2w_1w_2\sigma_1\sigma_2\rho$$

## Now we can plot our 2 assets in Mean Variance space and see how they relate to each other 

In [ ]:
#set up the graph space
Ax2 = bq.LinearScale()
Ay2 = bq.LinearScale()



assetline = bq.Scatter(x=[], y=[],
             scales={'x': Ax2, 'y': Ay2})
ax2_x = bq.Axis(scale=Ax2, label='Volatility', tick_format='0.2f')
ax2_y = bq.Axis(scale=Ay2, orientation='vertical', label='Expected Return')

MarkoPort = bq.Figure(marks=[assetline], axes=[ax2_x, ax2_y], title='2 Asset Portfolio', animation = 1000)

In [ ]:
corr_control = widgets.Dropdown(options=[1, .75,.5,.25,0,-.25,-.5, -.75, -1],
    value=0)
CorrOut =widgets.Output()
def UPdateCorr(*args):
    Asset_1_mu = 0.05
    Asset_2_mu = 0.15
    Asset_1_vol = 0.06
    Asset_2_vol = 0.20
    corr = corr_control.value
    Weights = [1, .75, .5, .25, 0]
    E_r = []
    Vols = []
    for weight in Weights:
        expRet = weight * Asset_1_mu + ((1-weight) * Asset_2_mu)
        E_r.append(expRet)
        Vol = np.sqrt((weight**2) * (Asset_1_vol**2) + ((1-weight)**2) * (Asset_2_vol**2) + (2*weight *(1-weight)*Asset_1_vol*Asset_2_vol*corr))
        Vols.append(Vol)
    
    with CorrOut:
        assetline.x = Vols
        assetline.y = E_r
        

In [ ]:
corr_control.observe(UPdateCorr,'value')

In [ ]:
Corrgraph = widgets.VBox([corr_control,MarkoPort])
display(Corrgraph)

## Now we can see what many assets can look like together

In [ ]:
stocks = ['HedgeFunds',	'RealAsset'	,'ACWI'	,'Bonds','Cash']
VolData = {'Vol': [0.0468,0.04975,0.1513,0.03401,0]}
RetData = {'Ret':[0.0652,0.0631,.059811,.0351,0.01]}

mean_returns = pd.DataFrame(RetData, columns = ['Ret'], index = stocks)
Vols = pd.DataFrame(VolData, columns = ['Vol'], index = stocks)

a = [1, 0.171631, 0.553449, 0.007903, 0]
b = [0.171631, 1.0 ,0.073338 ,-0.108612, 0]
c =[ 0.553449, 0.073338, 1.0, -0.026649, 0]
d= [0.007903,  -0.108612, -0.026649, 1.0, 0]
e = [0,0,0,0,1.0]
CorrMat = np.vstack([a,b,c,d,e])
cov_matrix  = pd.DataFrame(corr2cov(CorrMat,Vols), index = stocks, columns = stocks)

mean_returns =mean_returns.loc[:,'Ret']
ReturnScale = 1

In [ ]:
#set number of runs of random portfolio weights
num_portfolios = 10000


In [ ]:
#set up array to hold results
#We have increased the size of the array to hold the weight values for each stock
results = np.zeros((4+len(stocks)-1,num_portfolios))
 
for i in range(num_portfolios):
    #select random weights for portfolio holdings *No short selling
    weights = np.array(np.random.random(len(stocks)))
    
    #select random weights for portfolio holdings *short selling
    #weights = np.array(-1*np.random.random(len(stocks)) + .5)
    
    #rebalance weights to sum to 1
    weights /= np.sum(weights)
    
    #calculate portfolio return and volatility
    portfolio_return = np.sum(mean_returns * weights) * ReturnScale
    portfolio_std_dev = np.sqrt(np.dot(weights.T,np.dot(cov_matrix, weights))) * np.sqrt(ReturnScale)
 
    #store results in results array
    results[0,i] = portfolio_return
    results[1,i] = portfolio_std_dev
    #store Sharpe Ratio (return / volatility) - risk free rate element excluded for simplicity
    results[2,i] = results[0,i] / results[1,i]
    #iterate through the weight vector and add data to results array
    for j in range(len(weights)):
        results[j+3,i] = weights[j]

In [ ]:
#Label the result frame for the output and securities        
cols = ['ret','stdev','sharpe'] 
for stock in stocks:
    cols.append(stock)
    
#convert results array to Pandas DataFrame
results_frame = pd.DataFrame(results.T,columns=cols)
 
#locate position of portfolio with highest Sharpe Ratio
max_sharpe_port = results_frame.iloc[results_frame['sharpe'].idxmax()]
#locate positon of portfolio with minimum standard deviation
min_vol_port = results_frame.iloc[results_frame['stdev'].idxmin()]

In [ ]:
sc_x = bq.LinearScale()
sc_y = bq.LinearScale()

sc_c1 = bq.ColorScale()


scatter = bq.Scatter(x=results_frame.stdev*100, y=results_frame.ret*100, color=results_frame.sharpe,
                  scales={'x': sc_x, 'y': sc_y, 'color': sc_c1})

ax_y = bq.Axis(label='Expected Return %', scale=sc_y, 
            orientation='vertical', side='left')

ax_x = bq.Axis(label='Volatility', scale=sc_x, num_ticks=10, label_location='end')
ax_c = bq.ColorAxis(scale=sc_c1, tick_format='0.2', label='Sharpe Ratio', orientation='vertical', side='right')

m_chart = dict(top=50, bottom=70, left=50, right=100)

Graph = bq.Figure(axes=[ax_x, ax_c, ax_y], marks=[scatter],fig_margin=m_chart,
       title='Possible Portfolios') #, fig_margin=m_chart

In [ ]:
sc_c1.colors = ['blue', 'yellow', 'red']

In [ ]:
Graph